## Parsing + cleaning labelled training data 

In [26]:
import pandas as pd
import string
import apriori

In [27]:
labelled_data = []
with open("../resources/tagging_lexicons/Sentences_AllAgree.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()
positive_data = [(i.split('@')[0].strip(), i.split('@')[1].strip()) for i in labelled_data if '@positive' in i]  
negative_data = [(i.split('@')[0].strip(), i.split('@')[1].strip()) for i in labelled_data if '@negative' in i]
neutral_data = [(i.split('@')[0].strip(), i.split('@')[1].strip()) for i in labelled_data if '@neutral' in i]
total_label_data = positive_data + negative_data + neutral_data
labeled_df = pd.DataFrame(total_label_data, columns=["Sentence", "Label"])
labeled_df['Sentence'][0]


"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m ."

In [28]:
def cleanText(text):
    text = text.translate(str.maketrans (dict.fromkeys(string.punctuation)))
    text = text.lower()
    return text

In [29]:
#cleanText(labeled_df['Sentence'][0])
text= labeled_df['Sentence'][0]
#cleanText(labeled_df['Sentence'][0])
text= labeled_df['Sentence'][0]
dict.fromkeys(string.punctuation)
#text.translate(str.maketrans (dict.fromkeys(string.punctuation)))
#labeled_df['Sentence'] = labeled_df['Sentence'].apply(cleanText)
#text.translate(str.maketrans (dict.fromkeys(string.punctuation)))
#labeled_df['Sentence'] = labeled_df['Sentence'].apply(cleanText)

{'!': None,
 '"': None,
 '#': None,
 '$': None,
 '%': None,
 '&': None,
 "'": None,
 '(': None,
 ')': None,
 '*': None,
 '+': None,
 ',': None,
 '-': None,
 '.': None,
 '/': None,
 ':': None,
 ';': None,
 '<': None,
 '=': None,
 '>': None,
 '?': None,
 '@': None,
 '[': None,
 '\\': None,
 ']': None,
 '^': None,
 '_': None,
 '`': None,
 '{': None,
 '|': None,
 '}': None,
 '~': None}

## Loading tagging dictionaries

In [30]:
# Directionality
labelled_data = []
labelled_direction = []    
with open("../resources/tagging_lexicons/Directionality.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for pairs in labelled_data:
    labelled_direction += [(pairs[:-1].strip().split(" ")[0].strip(),pairs[:-1].strip().split(" ")[1].strip())]

# Positive
labelled_data = []
labelled_positive = []    
with open("../resources/tagging_lexicons/LM_positive.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for words in labelled_data:
    labelled_positive += [words[:-1].strip()]
    labelled_positive = [x.lower() for x in labelled_positive]
labelled_positive = sorted(labelled_positive) 

# Negative
labelled_data = []
labelled_negative = []    
with open("../resources/tagging_lexicons/LM_negative.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for words in labelled_data:
    labelled_negative += [words[:-1].strip()]
    labelled_negative = [x.lower() for x in labelled_negative]
labelled_negative = sorted(labelled_negative) 

# Leading
labelled_data = []
labelled_leading = []    
with open("../resources/tagging_lexicons/Leading Indicators.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   

for words in labelled_data:
    labelled_leading += [words[:-1].strip()]
labelled_leading = sorted(labelled_leading) 

# Lagging
labelled_data = []
labelled_lagging = []    
with open("../resources/tagging_lexicons/Lagging Indicators.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   

for words in labelled_data:
    labelled_lagging += [words[:-1].strip()]
labelled_lagging = sorted(labelled_lagging)

# Uncertainty
labelled_data = []
labelled_uncertainty = []    
with open("../resources/tagging_lexicons/LM_uncertainty.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()
for words in labelled_data:
    labelled_uncertainty += [words[:-1].strip().split(" ")]
final_labelled_uncertainty = []    
for i in labelled_uncertainty:
    final_labelled_uncertainty += i
    
final_labelled_uncertainty = [x.lower() for x in final_labelled_uncertainty][:-1]

# Constraining
labelled_data = []
labelled_constraining = []    
with open("../resources/tagging_lexicons/LM_constraining.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()
for words in labelled_data:
    labelled_constraining += [words[:-1].strip().split(" ")]
final_labelled_constraining = []    
for i in labelled_constraining:
    final_labelled_constraining += i
final_labelled_constraining = [x.lower() for x in final_labelled_constraining]

# Modal
labelled_data = []
labelled_modal = []    
with open("../resources/tagging_lexicons/modal.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for pairs in labelled_data:
    labelled_modal += [(pairs[:-1].strip().split(" ")[0].strip().lower(),pairs[:-1].strip().split(" ")[1].strip())]

In [6]:
word_lists_dict = {'LM_pos' :labelled_positive,'LM_neg' : labelled_negative ,'LagInd' : labelled_lagging ,'LeadInd' : labelled_leading ,'dir:UP' : [x[0] for x in labelled_direction if x[1] == 'UP'],'dir:DOWN': [x[0] for x in labelled_direction if x[1] == 'DOWN'], 'constrain' : final_labelled_constraining,'uncertain' : final_labelled_uncertainty,'mod:weak':[x[0] for x in labelled_modal if x[1] == 'WEAK'],'mod:medium': [x[0] for x in labelled_modal if x[1] == 'MEDIUM'],'mod:strong' : [x[0] for x in labelled_modal if x[1] == 'STRONG'] }

In [31]:
# Need to modify exisitng dictionary for new tagging function
# ? : Purpose of new_tag_data_structure
def new_tag_data_structure(tags_dict):
    new_tags = {}
    for category in tags_dict:
        temp_dict = {}
        first_words_list = [i.split(' ') for i in  tags_dict[category]]
        first_words_set = set([i[0] for i in  first_words_list])
        for i in first_words_set:
            for j in first_words_list:
                if j[0] == i:
                    if len(j) == 1:
                        temp_dict[i] = [[]]
                    else:
                        if i not in temp_dict:
                            temp_dict[i] = []
                        temp_dict[i].append(j[1:])
        new_tags[category] = temp_dict
    return new_tags

In [33]:
word_lists_dict = new_tag_data_structure(word_lists_dict)
word_lists_dict

{'LM_pos': {'innovate': [[]],
  'inventiveness': [[]],
  'progresses': [[]],
  'collaborated': [[]],
  'stronger': [[]],
  'enabling': [[]],
  'invention': [[]],
  'honorable': [[]],
  'delightfully': [[]],
  'better': [[]],
  'fantastic': [[]],
  'revolutionizes': [[]],
  'satisfactory': [[]],
  'assured': [[]],
  'distinctions': [[]],
  'profitability': [[]],
  'satisfaction': [[]],
  'satisfy': [[]],
  'enthusiastically': [[]],
  'advancements': [[]],
  'collaborate': [[]],
  'distinctiveness': [[]],
  'enhance': [[]],
  'complimented': [[]],
  'dream': [[]],
  'exclusives': [[]],
  'positive': [[]],
  'prestige': [[]],
  'succeeding': [[]],
  'satisfying': [[]],
  'great': [[]],
  'satisfied': [[]],
  'accomplished': [[]],
  'excels': [[]],
  'opportunity': [[]],
  'stabilized': [[]],
  'advantaged': [[]],
  'inventing': [[]],
  'upturns': [[]],
  'exciting': [[]],
  'strengthen': [[]],
  'honors': [[]],
  'efficiencies': [[]],
  'exclusively': [[]],
  'surpasses': [[]],
  'favorin

In [34]:
# dumping lexicons on pkl format
import pickle
#dumping apriori rules in pkl format
pickling_on_ = open("../resources/pickle/word_list_dict.pkl","wb")
pickle.dump(word_lists_dict, pickling_on_)
pickling_on_.close()

## Analysis 

In [35]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
import re
import string
import spacy
import operator
nlp = spacy.load("en_core_web_sm")

# Tagging sentences 

In [36]:
def tag_data(sent, new_tags, spacy_tokenizer):
    # replace words that belongs in lexicons with the lexicons title.
    # iterates through through each word in a sentence
    counter = 0
    # tokenize_text list of tokens in sentence
    tokenize_text = [str(word) for sentence in spacy_tokenizer(sent).sents for word in sentence]
    while counter < len(tokenize_text):
        #loops through lexicons titles in new_tags
        for category in new_tags:
            if tokenize_text[counter] in new_tags[category]:
                complete_matches = []
                #finds maximum match of the keys in the lexicon
                for word_list in new_tags[category][tokenize_text[counter]]:
                    complete_match = True
                    inner_counter = counter + 1
                    #loops through values that are associated with each key in lexicon
                    for word in word_list:
                        if len(word) == 0:
                            complete_matches.append(word_list)
                            continue
                        try:
                            if tokenize_text[inner_counter] == word:
                                complete_match = True
                            else:
                                complete_match = False
                                break
                        except:
                            complete_match = False
                            break
                        inner_counter += 1
                    if complete_match:
                        complete_matches.append(word_list)
                # check for complete matches and find length of maximum matched key        
                try:
                    if complete_matches == [[]]:
                        max_length = 1
                    else:
                        max_length =max([len(k) for k in complete_matches])
                except:
                    max_length = 0
                perfect_match = []
                # replace the matched with the respective lexicon title
                if max_length != 0:
                    if complete_matches !=[[]]:
                        perfect_match = [k for k in complete_matches if len(k) == max_length][0]
                        new_tokenize_text = tokenize_text[:counter] + ['<'+category+'>'] + tokenize_text[counter+len(perfect_match)+1:]
                    else:
                        new_tokenize_text = tokenize_text[:counter] + ['<'+category+'>'] + tokenize_text[counter+1:]
                    tokenize_text = new_tokenize_text.copy()
                #print(complete_matches, max_length, perfect_match)
        counter += 1
    return ' '.join(tokenize_text)

def remove_non_tag_word(sent):
    tags = []
    for i in sent.split(' '):
        if '<' in i and '>' in i:
            tags.append(i)
    return tags

In [37]:
import time
t0 = time.time()
labeled_df['Tagged_sentence'] = labeled_df.Sentence.apply(lambda x: tag_data(x,word_lists_dict,nlp))
labeled_df['Tagged'] = labeled_df.Tagged_sentence.apply(remove_non_tag_word)
t1 = time.time()
print('time to tag: ' + str(t1 - t0))

time to tag: 42.27311611175537


## Implementing Apriori Algorithm, training generating association rules, pickling

In [38]:
# I didn't notice any "False" flags here, does this happen at all?

# Remove empty tagged rows
labeled_df['len'] = labeled_df.Tagged.apply(lambda x: len(x) >0)
labeled_df = labeled_df[labeled_df.len]

labeled_df['tagged_label'] = labeled_df.Tagged + labeled_df.Label.apply(lambda x: list([x]))

# dropping the temporary length column created
labeled_df = labeled_df.drop(['len'], axis = 1)

In [39]:
# What's the difference between Tagged and tagged_lable
labeled_df


,Sentence,Label,Tagged_sentence,Tagged,tagged_label
0,"For the last quarter of 2010 , Componenta 's n...",positive,"For the last quarter of 2010 , Componenta 's <...","[<LagInd>, <LagInd>, <dir:UP>, <constrain>, <c...","[<LagInd>, <LagInd>, <dir:UP>, <constrain>, <c..."
1,"In the third quarter of 2010 , net sales incre...",positive,"In the third quarter of 2010 , <LagInd> <LagIn...","[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI...","[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI..."
2,Operating profit rose to EUR 13.1 mn from EUR ...,positive,Operating <LagInd> <dir:UP> to EUR 13.1 mn fro...,"[<LagInd>, <dir:UP>, <LagInd>, <LagInd>]","[<LagInd>, <dir:UP>, <LagInd>, <LagInd>, posit..."
3,"Operating profit totalled EUR 21.1 mn , up fro...",positive,"Operating <LagInd> totalled EUR 21.1 mn , <dir...","[<LagInd>, <dir:UP>, <LagInd>, <LagInd>]","[<LagInd>, <dir:UP>, <LagInd>, <LagInd>, posit..."
4,Finnish Talentum reports its operating profit ...,positive,Finnish Talentum reports its <LagInd> <LagInd>...,"[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI...","[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI..."
5,Clothing retail chain Sepp+ñl+ñ 's sales incre...,positive,Clothing retail chain Sepp+ñl+ñ 's <LagInd> <d...,"[<LagInd>, <dir:UP>, <LagInd>, <LagInd>, <dir:...","[<LagInd>, <dir:UP>, <LagInd>, <LagInd>, <dir:..."
6,Consolidated net sales increased 16 % to reach...,positive,Consolidated <LagInd> <LagInd> <dir:UP> 16 % t...,"[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI...","[<LagInd>, <LagInd>, <dir:UP>, <LagInd>, <LagI..."
7,Foundries division reports its sales increased...,positive,Foundries division reports its <LagInd> <dir:U...,"[<LagInd>, <dir:UP>, <LagInd>, <dir:UP>]","[<LagInd>, <dir:UP>, <LagInd>, <dir:UP>, posit..."
8,"HELSINKI ( AFX ) - Shares closed higher , led ...",positive,"HELSINKI ( AFX ) - Shares <LM_neg> <dir:UP> , ...","[<LM_neg>, <dir:UP>, <dir:UP>, <LagInd>, <dir:...","[<LM_neg>, <dir:UP>, <dir:UP>, <LagInd>, <dir:..."
9,Its board of directors will propose a dividend...,positive,Its board of directors <mod:strong> propose <c...,"[<mod:strong>, <constrain>, <LagInd>, <dir:UP>]","[<mod:strong>, <constrain>, <LagInd>, <dir:UP>..."


### Splitting to training and testing 

In [50]:
frac_eigth = labeled_df.sample(frac=0.8)

index_to_delete_list = frac_eigth.index.tolist()

test_data = labeled_df.drop(index_to_delete_list,axis = 0)

### Training apriori model

In [51]:
from efficient_apriori import apriori
itemsets, rules = apriori(frac_eigth.tagged_label.get_values().tolist(), min_support=0,  min_confidence=0)

# Parsing association rules
def parsing_apriori_results(x,labels):
    final_rules = []
    for label in labels:  
        for pairs in x:
            pairs_ = re.findall('\{(.*?)\}',str(pairs))
            if pairs_[1] == label:
                final_rules += [[pairs_[0],pairs_[1],pairs.confidence]]
    return final_rules 

def dict_parsing_apriori_results(parsing_apriori):
    final_dict = {}
    for i in parsing_apriori:
        if i[0] not in final_dict:
            final_dict[i[0]] = {i[1] : i[2]}
        else:
            final_dict[i[0]][i[1]] = i[2]
    return final_dict

In [52]:
trial = parsing_apriori_results(rules,['negative','positive','neutral'])
trained_apriori_trial = dict_parsing_apriori_results(trial)
#predict sentiment based on tags fed.
def predict(tags,trained_apriori):
    for i in trained_apriori:
        #print(i.split(', '))
        if set(tags) == set(i.split(', ')):
            result = max(trained_apriori[i].items(), key=operator.itemgetter(1))
            return result[0]    

In [53]:
import pickle
#dumping apriori rules in pkl format after training
pickling_on = open("../resources/pickle/apriori_rules.pkl","wb")
pickle.dump(trained_apriori_trial, pickling_on)
pickling_on.close()


In [54]:
#loading pkl
pickle_off = open("../resources/pickle/apriori_rules.pkl","rb")
trained_apriori_trial = pickle.load(pickle_off)
#predict
test_data['predicted_label'] = test_data.Tagged.apply(lambda x: predict(x,trained_apriori_trial))
test_data['predict_result'] = test_data.Label == test_data.predicted_label

In [55]:
trained_apriori_trial

{'<LM_neg>': {'negative': 0.41452991452991456,
  'positive': 0.2606837606837607,
  'neutral': 0.3247863247863248},
 '<LM_pos>': {'negative': 0.058394160583941604,
  'positive': 0.583941605839416,
  'neutral': 0.35766423357664234},
 '<LagInd>': {'negative': 0.2607449856733524,
  'positive': 0.4283667621776504,
  'neutral': 0.3108882521489971},
 '<LeadInd>': {'negative': 0.06613226452905811,
  'positive': 0.1843687374749499,
  'neutral': 0.749498997995992},
 '<constrain>': {'negative': 0.18818380743982493,
  'positive': 0.2844638949671772,
  'neutral': 0.5273522975929978},
 '<dir:DOWN>': {'negative': 0.8571428571428571,
  'positive': 0.09142857142857143,
  'neutral': 0.05142857142857143},
 '<dir:UP>': {'negative': 0.036931818181818184,
  'positive': 0.8522727272727273,
  'neutral': 0.11079545454545454},
 '<mod:medium>': {'negative': 0.10204081632653061,
  'positive': 0.12244897959183673,
  'neutral': 0.7755102040816326},
 '<mod:strong>': {'negative': 0.0546875,
  'positive': 0.11328125,


In [56]:
test_data[["Label", "predicted_label"]] = test_data[["Label", "predicted_label"]].astype(str)
from sklearn.metrics import classification_report
print(classification_report(test_data.Label,test_data.predicted_label))

              precision    recall  f1-score   support

        None       0.00      0.00      0.00         0
    negative       0.64      0.94      0.76        64
     neutral       0.94      0.73      0.82       201
    positive       0.69      0.75      0.72       115

   micro avg       0.77      0.77      0.77       380
   macro avg       0.57      0.60      0.57       380
weighted avg       0.81      0.77      0.78       380



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Predict on new pdf 

In [61]:
import sys
sys.path.insert(0, '../services')
import pdf_parser
import pre_processing
test_json = pdf_parser.pdf_parse('../resources/test_pdfs/CM Q1-F19.pdf')
sentence_paragraph_df = pre_processing.parse_earnings_file(test_json)
sentence_df = sentence_paragraph_df['sentences']
sentence_df['greeting'] = sentence_df.sentences.apply(pre_processing.remove_greetings_filter)
sentence_df = sentence_df[sentence_df['greeting'] == False].drop(columns = ['greeting'])

In [62]:
from apriori import Apriori_Model
sentence_df['label'] = Apriori_Model().predict(sentence_df['sentences'])

In [63]:
sentence_df

,sentences,section,label
1,"Before we start, I'd like to update you on som...",mds,None
2,"Last December, Amy South assumed responsibilit...",mds,neutral
3,"So, I'd just like to take this opportunity to ...",mds,positive
4,Many of you on this call have already had intr...,mds,positive
5,"So, with that, let me turn the call over to Hr...",mds,None
6,"Thank you, Kevin.",mds,None
8,"Victor Dodig, CIBC's President and Chief Execu...",mds,neutral
9,"Kevin Glass, our Chief Financial Officer, will...",mds,neutral
10,We are joined in the room by CIBC business lea...,mds,None
11,They will be available to answer questions fol...,mds,neutral
